

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/NER_FR.ipynb)




# **Detect entities in French text**

## 1. Colab Setup

In [1]:
# Install Java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install SparkNLP
! pip install --ignore-installed spark-nlp

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode, sharing)
     |████████████████████████████████| 215.7MB 56kB/s 
     |████████████████████████████████| 204kB 42.5MB/s 
     |████████████████████████████████| 133kB 2.7MB/s 


## 2. Start the Spark session

In [2]:
import os
import json
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"

import pandas as pd
import numpy as np

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

spark = sparknlp.start()

## 3. Select the DL model

In [3]:
# If you change the model, re-run all the cells below.
# Applicable models: wikiner_840B_300
MODEL_NAME = "wikiner_840B_300"

## 4. Some sample examples

In [13]:
# Enter examples to be transformed as strings in this list
text_list = [
    """William Henry Gates III (né le 28 octobre 1955) est un magnat des affaires, développeur de logiciels, investisseur et philanthrope américain. Il est surtout connu comme le co-fondateur de Microsoft Corporation. Au cours de sa carrière chez Microsoft, Gates a occupé les postes de président, chef de la direction (PDG), président et architecte logiciel en chef, tout en étant le plus grand actionnaire individuel jusqu'en mai 2014. Il est l'un des entrepreneurs et pionniers les plus connus du révolution des micro-ordinateurs des années 1970 et 1980. Né et élevé à Seattle, Washington, Gates a cofondé Microsoft avec son ami d'enfance Paul Allen en 1975, à Albuquerque, au Nouveau-Mexique; il est devenu la plus grande société de logiciels informatiques au monde. Gates a dirigé l'entreprise en tant que président-directeur général jusqu'à sa démission en tant que PDG en janvier 2000, mais il est resté président et est devenu architecte logiciel en chef. À la fin des années 1990, Gates avait été critiqué pour ses tactiques commerciales, considérées comme anticoncurrentielles. Cette opinion a été confirmée par de nombreuses décisions de justice. En juin 2006, Gates a annoncé qu'il passerait à un poste à temps partiel chez Microsoft et à un emploi à temps plein à la Bill & Melinda Gates Foundation, la fondation caritative privée que lui et sa femme, Melinda Gates, ont créée en 2000. [ 9] Il a progressivement transféré ses fonctions à Ray Ozzie et Craig Mundie. Il a démissionné de son poste de président de Microsoft en février 2014 et a assumé un nouveau poste de conseiller technologique pour soutenir le nouveau PDG Satya Nadella.""",
    """La Joconde est une peinture à l'huile du XVIe siècle créée par Léonard. Il se tient au Louvre à Paris."""
]

## 5. Define Spark NLP pipeline

In [14]:
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

# The wikiner_840B_300 is trained with glove_840B_300, so the embeddings in the
# pipeline should match. Same applies for the other available models.
if MODEL_NAME == "wikiner_840B_300":
    embeddings = WordEmbeddingsModel.pretrained('glove_840B_300', lang='xx') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')
elif MODEL_NAME == "wikiner_6B_300":
    embeddings = WordEmbeddingsModel.pretrained('glove_6B_300', lang='xx') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')
elif MODEL_NAME == "wikiner_6B_100":
    embeddings = WordEmbeddingsModel.pretrained('glove_100d') \
        .setInputCols(['document', 'token']) \
        .setOutputCol('embeddings')

ner_model = NerDLModel.pretrained(MODEL_NAME, 'fr') \
    .setInputCols(['document', 'token', 'embeddings']) \
    .setOutputCol('ner')

ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

nlp_pipeline = Pipeline(stages=[
    document_assembler, 
    tokenizer,
    embeddings,
    ner_model,
    ner_converter
])

glove_840B_300 download started this may take some time.
Approximate size to download 2.3 GB
[OK!]
wikiner_840B_300 download started this may take some time.
Approximate size to download 14.4 MB
[OK!]


## 6. Run the pipeline

In [15]:
empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
result = pipeline_model.transform(df)

## 7. Visualize results

In [16]:
result.select(
    F.explode(
        F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')
    ).alias("cols")
).select(
    F.expr("cols['0']").alias('chunk'),
    F.expr("cols['1']['entity']").alias('ner_label')
).show(truncate=False)

+-------------------------------+---------+
|chunk                          |ner_label|
+-------------------------------+---------+
|William Henry Gates III        |PER      |
|Microsoft Corporation          |ORG      |
|Microsoft                      |ORG      |
|Gates                          |PER      |
|Né et élevé à Seattle          |MISC     |
|Washington                     |LOC      |
|Gates                          |PER      |
|Microsoft                      |ORG      |
|Paul Allen                     |PER      |
|Albuquerque                    |LOC      |
|Nouveau-Mexique                |LOC      |
|Gates                          |PER      |
|Gates                          |PER      |
|Gates                          |PER      |
|Microsoft                      |ORG      |
|Bill & Melinda Gates Foundation|ORG      |
|Melinda Gates                  |PER      |
|Ray Ozzie                      |PER      |
|Craig Mundie                   |PER      |
|Microsoft                      